<a href="https://colab.research.google.com/github/PieterDujardin/NER/blob/master/ColabNotebooks/i2b2_2010/CRF_scikitlearn_on_i2b22010.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://pypi.org/project/python-crfsuite/

# import statements


In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')
# import spacy
# nlp = spacy.load("en_core_web_sm")
# nlp = spacy.load("es_core_news_sm")

In [0]:
!pip install sklearn_crfsuite
!pip install seqeval

In [0]:
from itertools import chain

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split



import sklearn_crfsuite
from sklearn_crfsuite import scorers


# from sklearn.metrics import f1_score

from seqeval.metrics import classification_report,accuracy_score,f1_score

from sklearn_crfsuite import metrics

# function definitions

In [0]:
def loadandformat(datasets):

    # data=[]
    # for dataset in datasets:
    #   # the training, dev and test set in one file
    #   with open('drive/My Drive/Colab Notebooks/BERTwithNER3/conll2003format/{}'.format(dataset),'r') as f:
    #       data +=  f.readlines()
    sentence=[]
    sentences=[]
    for dataset in datasets:
      with open('drive/My Drive/Colab Notebooks/#2_BERT_on_i2b2_2010/data/conll2003format/{}'.format(dataset),'r') as f:
        for line in f:
          if line == '\n':
              sentences.append(sentence)
              sentence=[]
          else:
              sentence.append(line)

    for sent in range(len(sentences)):
      for word in range(len(sentences[sent])):
          sentences[sent][word] = str(sentences[sent][word]).replace('\t',' ')
          sentences[sent][word] = str(sentences[sent][word]).rstrip('\n')
          sentences[sent][word]= str(sentences[sent][word]).split()

    sentenceslabels =[]
    sentencestext =[]

    placeholder0=[]
    placeholder1=[]

    for sent in range(len(sentences)):
      for word in range(len(sentences[sent])):
          placeholder0.append(sentences[sent][word][1])
          placeholder1.append(sentences[sent][word][0])
          
      sentenceslabels.append(placeholder0)
      sentencestext.append(placeholder1)
      placeholder0=[]
      placeholder1=[]

    joinedlist =[]
    for i in range(len(sentenceslabels)-1):
      joinedlist.extend(sentenceslabels[i])

    tags_vals = list(set(joinedlist))

    tags_vals.append('X')
    tags_vals.append('[CLS]')
    tags_vals.append('[SEP]')

    tag2idx = {t: i for i, t in enumerate(tags_vals)}
    tag2name={tag2idx[key] : key for key in tag2idx.keys()}


    return (sentencestext,sentenceslabels,tags_vals,tag2idx,tag2name)

In [0]:
def word2features(sent, i):
    word = sent[i]
#     postag = sent[i][1]
#     doc = nlp(word)
    
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        # 'postag': postag,
        # 'postag[:2]': postag[:2],
        'prefix_1': word[0],
        'prefix_2': word[:2],
        'prefix_3': word[:3],
        'prefix_4': word[:4],
        'suffix_5': word[-1:],
        'suffix_6': word[-2:],
        'suffix_7': word[-3:],
        'suffix_8': word[-4:],
        'word_has_hyphen': 1 if '-' in word else 0,
        'token_length': len(word),
        'position_in_sentence': i,
        'fractional_position': i/len(sent),
#         'which_bin_in_sentence0': 1 if (i/len(sent)) <= 0.25 else 0,
#         'which_bin_in_sentence1': 1 if  0.25 <= i/len(sent) <=0.5 else 0,
#         'which_bin_in_sentence2': 1 if 0.5 <= i/len(sent) <=0.75 else 0,
#         'which_bin_in_sentence3': 1 if 0.75<= i/len(sent) else 0,
        
#         'spacy_01': doc[0].tag_,
#         'spacy_02': doc[0].dep_,
#         'spacy_03': doc[0].shape_,
#         'spacy_04': doc[0].like_num,
#         'spacy_05': doc[0].lemma_,
#         'spacy_06': doc[0].is_stop,
#         'spacy_07': doc[0].is_bracket,
#         'spacy_08': doc[0].is_alpha, 
    }
    
    
    if i > 0:
        word1 = sent[i-1]
#         postag1 = sent[i-1][1]
#         doc1= nlp(word1)
        
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
#             '-1:postag': postag1,
#             '-1:postag[:2]': postag1[:2],
            
#             '-1:spacy_01': doc1[0].tag_,
#             '-1:spacy_02': doc1[0].dep_,
#             '-1:spacy_03': doc1[0].shape_,
#             '-1:spacy_04': doc1[0].like_num,
#             '-1:spacy_05': doc1[0].lemma_,
#             '-1:spacy_06': doc1[0].is_stop,
#             '-1:spacy_07': doc1[0].is_bracket,
#             '-1:spacy_08': doc1[0].is_alpha, 
     
        })
    else:
        features['BOS'] = True
        
        
    if i < len(sent)-1:
        word1 = sent[i+1]
#         postag1 = sent[i+1][1]
#         doc2= nlp(word1)
        
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
#             '+1:postag': postag1,
#             '+1:postag[:2]': postag1[:2],
            
#             '+1:spacy_01': doc2[0].tag_,
#             '+1:spacy_02': doc2[0].dep_,
#             '+1:spacy_03': doc2[0].shape_,
#             '+1:spacy_04': doc2[0].like_num,
#             '+1:spacy_05': doc2[0].lemma_,
#             '+1:spacy_06': doc2[0].is_stop,
#             '+1:spacy_07': doc1[0].is_bracket,
#             '+1:spacy_08': doc1[0].is_alpha, 
            
            
        })
    else:
        features['EOS'] = True
                
    if i > 1:  
        word2 = sent[i-2]
#         postag2 = sent[i-2][1]
        
        features.update({
            '-2:word.lower()': word2.lower(),
            '-2:word.istitle()': word2.istitle(),
            '-2:word.isupper()': word2.isupper(),
#             '-2:postag': postag2,
#             '-2:postag[:2]': postag2[:2],
             })
        
    if i < len(sent)-2:
        word2 = sent[i+2]
#         postag2 = sent[i+2][1]
        
        features.update({
            '+2:word.lower()': word2.lower(),
            '+2:word.istitle()': word2.istitle(),
            '+2:word.isupper()': word2.isupper(),
#             '+2:postag': postag2,
#             '+2:postag[:2]': postag2[:2],
        })
            
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

# function calls

In [0]:
#load data: 'ortho1.txt' , 'ortho2.txt', 'cardio.txt'
(sentences,labels,tags_vals,tag2idx,tag2name) = loadandformat(['train.txt','dev.txt','test.txt'])

In [0]:
train_sents, test_sents, y_train, y_test= train_test_split(sentences,labels, random_state=1, test_size=0.25)

In [0]:
train_sents, dev_sents, y_train, y_dev= train_test_split(train_sents,y_train, random_state=1, test_size=0.2)

This is what word2features extracts:

In [0]:
# word2features(train_sents[0], 0)

# word2features(sentences[0], 0)

# sent2features(train_sents[0])

Extract features from the data:

In [0]:
# %%time
# X_train = [sent2features(s) for s in train_sents]
# y_train = [sent2labels(s) for s in train_sents]

# X_test = [sent2features(s) for s in test_sents]
# y_test = [sent2labels(s) for s in test_sents]

In [0]:
X_train = [sent2features(s) for s in train_sents]
X_dev = [sent2features(s) for s in dev_sents]
X_test = [sent2features(s) for s in test_sents]

In [0]:
# X_train[0],y_train[0]

## Training

To see all possible CRF parameters check its docstring. Here we are useing L-BFGS training algorithm (it is default) with Elastic Net (L1 + L2) regularization.

In [0]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.5, 
    c2=0.3, 
    max_iterations=100, 
    all_possible_transitions=True
)
crf.fit(X_train, y_train, X_dev=X_dev, y_dev= y_dev)

## Evaluation

There is much more O entities in data set, but we're more interested in other entities. To account for this we'll use averaged F1 score computed for all labels except for O. ``sklearn-crfsuite.metrics`` package provides some useful metrics for sequence classification task, including this one.

In [0]:
labels = list(crf.classes_)
labels.remove('O')
labels

In [0]:
y_pred = crf.predict(X_test) 

## F1-calculation 1 - sklearn_crfsuite.metrics

In [0]:
from sklearn_crfsuite import metrics

In [0]:
#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html
#micro macro weighted
metrics.flat_f1_score(y_test, y_pred,average='micro',labels=labels) 

Inspect per-class results in more detail:

In [0]:
# group B and I results
sorted_labels = sorted(
    labels, 
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred,labels=sorted_labels, digits=3
)) 

## F1-calculation 2 - sequal metrics


In [0]:
from seqeval.metrics import classification_report,accuracy_score,f1_score


In [0]:
f1_score

In [0]:
f1_score(y_test,y_pred)

In [0]:
report = classification_report(y_test, y_pred,digits=4)
print(report)

## F1-calc 3 -

In [0]:
from sklearn.metrics import f1_score,classification_report

In [0]:
f1_score

In [0]:
## flatten in order to use this

In [0]:
# f1_score(y_test,y_pred, average='micro', labels=labels)

In [0]:
print(classification_report(y_test,y_pred,labels=labels ))

# predictions

In [0]:
a =0

test_sents[a],y_test[a],y_pred[a]

In [0]:
a = 0
print( '\n')
print(test_sents[a])
print('\033[1m' + "Labels:      " + '\033[0m' + str(y_test[a]))
print('\033[1m' + "Predictions: "+ '\033[0m' + str(y_pred[a]))
print('\n')

In [0]:
a = 2
print( '\n')
print(test_sents[a])
print('\033[1m' + "Labels:      " + '\033[0m' + str(y_test[a]))
print('\033[1m' + "Predictions: "+ '\033[0m' + str(y_pred[a]))
print('\n')

In [0]:
a = 4
print( '\n')
print(test_sents[a])
print('\033[1m' + "Labels:      " + '\033[0m' + str(y_test[a]))
print('\033[1m' + "Predictions: "+ '\033[0m' + str(y_pred[a]))
print('\n')

In [0]:
a = 5
print( '\n')
print(test_sents[a])
print('\033[1m' + "Labels:      " + '\033[0m' + str(y_test[a]))
print('\033[1m' + "Predictions: "+ '\033[0m' + str(y_pred[a]))
print('\n')

In [0]:
a = 8
print( '\n')
print(test_sents[a])
print('\033[1m' + "Labels:      " + '\033[0m' + str(y_test[a]))
print('\033[1m' + "Predictions: "+ '\033[0m' + str(y_pred[a]))
print('\n')

In [0]:
a = 9
print( '\n')
print(test_sents[a])
print('\033[1m' + "Labels:      " + '\033[0m' + str(y_test[a]))
print('\033[1m' + "Predictions: "+ '\033[0m' + str(y_pred[a]))
print('\n')

In [0]:
a = 10
print( '\n')
print(test_sents[a])
print('\033[1m' + "Labels:      " + '\033[0m' + str(y_test[a]))
print('\033[1m' + "Predictions: "+ '\033[0m' + str(y_pred[a]))
print('\n')

In [0]:
a = 13
print( '\n')
print(test_sents[a])
print('\033[1m' + "Labels:      " + '\033[0m' + str(y_test[a]))
print('\033[1m' + "Predictions: "+ '\033[0m' + str(y_pred[a]))
print('\n')

In [0]:
a = 14
print( '\n')
print(test_sents[a])
print('\033[1m' + "Labels:      " + '\033[0m' + str(y_test[a]))
print('\033[1m' + "Predictions: "+ '\033[0m' + str(y_pred[a]))
print('\n')

In [0]:
a = 15
print( '\n')
print(test_sents[a])
print('\033[1m' + "Labels:      " + '\033[0m' + str(y_test[a]))
print('\033[1m' + "Predictions: "+ '\033[0m' + str(y_pred[a]))
print('\n')

In [0]:
a = 19
print( '\n')
print(test_sents[a])
print('\033[1m' + "Labels:      " + '\033[0m' + str(y_test[a]))
print('\033[1m' + "Predictions: "+ '\033[0m' + str(y_pred[a]))
print('\n')

## Hyperparameter Optimization


In [0]:
%%time
# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    max_iterations=100, 
    all_possible_transitions=True)

params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score, 
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space, 
                        cv=3, 
                        verbose = 2, 
                        n_jobs = -1, 
                        n_iter = 20, 
                        scoring = f1_scorer)

rs.fit(X_train, y_train)

Best result:

In [0]:
# crf = rs.best_estimator_
print('best params:', rs.best_params_)
print('best CV score:', rs.best_score_)
print('model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

### Check parameter space



In [0]:
_x = [s.parameters['c1'] for s in rs.grid_scores_]
_y = [s.parameters['c2'] for s in rs.grid_scores_]
_c = [s.mean_validation_score for s in rs.grid_scores_]

fig = plt.figure()
fig.set_size_inches(12, 12)
ax = plt.gca()
ax.set_yscale('log')
ax.set_xscale('log')
ax.set_xlabel('C1')
ax.set_ylabel('C2')
ax.set_title("Randomized Hyperparameter Search CV Results (min={:0.3}, max={:0.3})".format(
    min(_c), max(_c)
))

ax.scatter(_x, _y, c=_c, s=60, alpha=0.9, edgecolors=[0,0,0])

print("Dark blue => {:0.4}, dark red => {:0.4}".format(min(_c), max(_c)))

## check best estimator on our test data



In [0]:
crf = rs.best_estimator_
y_pred = crf.predict(X_test)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

##check what classifier learned

In [0]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

In [0]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))    

print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))

print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])